In [4]:
import numpy as np
import torch
import torch.nn as nn
from IPython.display import Audio
import librosa
from models.codec.amphion_codec.codec import CodecEncoder, CodecDecoder
from utils.util import load_config

In [12]:
cfg = load_config("egs/codec/codec_dac_vocos_16K/exp_config_base.json")
print(cfg)

{'model_type': 'Codec', 'dataset': [''], 'preprocess': {'hop_size': 320, 'sample_rate': 16000, 'max_length': 24000, 'processed_dir': '', 'valid_file': 'valid.json', 'train_file': 'train.json'}, 'model': {'encoder': {'d_model': 96, 'up_ratios': [4, 4, 4, 5], 'out_channels': 256, 'use_tanh': False}, 'decoder': {'in_channel': 256, 'upsample_initial_channel': 1536, 'up_ratios': [5, 4, 4, 4], 'num_quantizers': 8, 'codebook_size': 1024, 'codebook_dim': 8, 'quantizer_type': 'fvq', 'quantizer_dropout': 0.5, 'commitment': 0.25, 'codebook_loss_weight': 1.0, 'use_l2_normlize': True, 'codebook_type': 'euclidean', 'kmeans_init': False, 'kmeans_iters': 10, 'decay': 0.8, 'eps': 0.5, 'threshold_ema_dead_code': 2, 'weight_init': False, 'use_vocos': True, 'vocos_dim': 512, 'vocos_intermediate_dim': 4096, 'vocos_num_layers': 24, 'n_fft': 1280, 'hop_size': 320, 'padding': 'same'}, 'period_gan': {'max_downsample_channels': 512, 'channels': 32, 'channel_increasing_factor': 2}, 'spec_gan': {'stft_params': {'

In [18]:
encoder = CodecEncoder(cfg=cfg.model.encoder)
decoder = CodecDecoder(cfg=cfg.model.decoder)

In [21]:
model_path = "/ckpts/codec_16k_320hopsize_8layer_vocos/pytorch_model.bin"
checkpoint = torch.load(model_path, map_location="cpu")
encoder.load_state_dict(checkpoint)
model_path = "/ckpts/codec_16k_320hopsize_8layer_vocos/pytorch_model_1.bin"
checkpoint = torch.load(model_path, map_location="cpu")
decoder.load_state_dict(checkpoint)

<All keys matched successfully>

In [22]:
test_wav_path = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/39/121916/39_121916_000002_000002.wav"
wav, sr = librosa.load(test_wav_path, sr=16000)
Audio(wav, rate=sr)

In [23]:
audio = torch.from_numpy(wav).unsqueeze(0)
print(audio.shape)

# encode the audio to latent
vq_emb = encoder(audio.unsqueeze(0))
print(vq_emb.shape)

vq_post_emb, vq_id, _, _, _ = decoder(
    vq_emb, vq=True, eval_vq=True
)
# latent after vq
print(vq_post_emb.shape)
# vq id
print(vq_id.shape)

torch.Size([1, 236960])


torch.Size([1, 256, 740])
torch.Size([1, 256, 740])
torch.Size([8, 1, 740])


In [24]:
recovered_audio = decoder(vq_post_emb, vq=False)
print(recovered_audio.shape)
recovered_audio = recovered_audio.squeeze(0).squeeze(0).detach().numpy()
Audio(recovered_audio, rate=sr)

torch.Size([1, 1, 236800])
